In [3]:
from collections import defaultdict
import numpy as np

In [4]:

# helper function that computes jaccard index of two set# helper 
def edge_weight(first_set, second_set):
    """ Computes edge_weight of two sets
        Arguments:
          first_set(set):
          second_set(set):
        Returns:
          index(float): edge_weight between two movies in movie network; it is between 0.0 and 1.0
    """
    index = 1.0
    if first_set or second_set:
        index = (float(len(first_set.intersection(second_set))) / len(first_set.union(second_set)))

    return index

In [21]:
actor2movie=defaultdict(set)
movie2actor=defaultdict(set)

filenames = ['actor_movies.txt', 'actress_movies.txt']
#with open('C:/Users/ektam/Desktop/Ekta/Subjects/Q3/EE232/Project4/project4_data/merged.txt', 'w') as outfile:
#with open('C:/Users/ektam/Desktop/Ekta/Subjects/Q3/EE232/Project4/project4_data/merged.txt', 'w') as outfile:
for fname in filenames:
    with open(fname) as infile:
        for line in infile:
            #words=line.split("\t\t")
            if(line.count("\t\t")>10):
                #line=line.strip()
                words=line.strip().split("\t\t")
                actor=(words[0]).replace(" ","")
                movielist=words[1:]
                for m in movielist:
                    m=m.split(")")[0].replace(" ","")
                    actor2movie[actor].add(m+")")
                    movie2actor[m+")"].add(actor)
                    

In [22]:
#To consider only movies with at least 5 actors/actresses
movie2actor = {movie : actors for (movie, actors) in movie2actor.iteritems() if len(actors) > 4}

In [23]:
#To maintain consistency in a2m
clean_movies=set(filter(lambda x :len(movie2actor[x])>4, movie2actor.keys()))
len(clean_movies)

197377

In [24]:
actor2movie = {actor : movies.intersection(clean_movies) for (actor, movies) in actor2movie.iteritems()}
len(actor2movie)

101128

In [8]:
#create and dump pickles for these 2 dictionaries
import pickle


pickle.dump(actor2movie, open("actor2movie.pkl", "wb" ))




In [25]:
pickle.dump(movie2actor, open("movie2actor.pkl", "wb" ))

In [26]:
A2M = pickle.load(open("actor2movie.pkl", "rb" ))
M2A = pickle.load(open("movie2actor.pkl", "rb" ))

In [27]:
len(A2M)

101128

In [28]:
len(M2A)

197377

In [29]:
moviemap={}
actormap={}
i=0
for m  in M2A.keys():
    moviemap[m]=i
    i=i+1
i=0
for a in A2M.keys():
    actormap[a]=i
    i=i+1

In [31]:
# create movie edge list for movie with at least 4 actors/actresses
count = 0
outfile = open("imdb_edgelist.txt", "w")
#outfile = open("temp1.txt", "w") with tab
#outfile = open("temp2.txt", "w")
stopper=0
processed_movie = set()
for i, actors in movie2actor.iteritems():
    # for a movie i, get the list of movies that share some actors
    #if stopper>5:
     #   break
    #stopper=stopper+1
    unique_movies = set()
    for j in actors:
        #print i
        #print actors
        #print j
        for k in actor2movie[j]:
            if k in processed_movie:
                continue
            weight = edge_weight(actors, movie2actor[k])
            unique_movies.add( (k, weight) )
    for (k, weight) in unique_movies:
        line = str(moviemap[i]) + '\t' + str(moviemap[k]) + '\t' + str( "%.3f" % weight) +'\n'
        outfile.write(line)
        count = count + 1
    processed_movie.add(i)
outfile.close()
print(count)

65933286
